<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #5: Add new frames to a Zooniverse workflow</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Dec 13th, 2021</h5>

# Set up and requirements

### Import Python packages

In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.server_utils as s_utils
import kso_utils.t5_utils as t5
import kso_utils.zooniverse_utils as zoo

print("Packages loaded successfully")

<IPython.core.display.Javascript object>

Packages loaded successfully


### Choose your project

In [46]:
project = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'), value='Koster_Sea…

### Initiate sql and zoo project

In [47]:
# Initiate db
db_info_dict = t_utils.initiate_db(project.value)

Enter your username for SNIC server········
Enter your password for SNIC server········
Updated sites
Updated movies
Updated species


In [48]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project.value)

Enter your Zooniverse user········
Enter your Zooniverse password········


# Retrieve Zooniverse information

In [49]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(project_name = project.value, 
                                                    db_info_dict = db_info_dict,
                                                    zoo_project = zoo_project,
                                                    zoo_info = ["subjects", "classifications", "workflows"])

Retrieving subjects from Zooniverse
subjects were retrieved successfully
Retrieving classifications from Zooniverse
classifications were retrieved successfully
Retrieving workflows from Zooniverse
workflows were retrieved successfully
Updated subjects
The database has a total of 2342 frame subjects and 7362 clip subjects have been updated


# Retrieve frames

## Select the species of interest

In [50]:
# Specify the species of interest
species_i = t5.choose_species(db_info_dict["db_path"])

SelectMultiple(description='Species', index=(0,), options=('Angular crab', 'Ascidians (any species)', 'Bivalve…

### Create the frames

In [107]:
frames_to_upload_df = t5.get_frames(species_ids = species_i.value, 
                                    db_path = db_info_dict["db_path"],
                                    zoo_info_dict = zoo_info_dict,
                                    server_dict = db_info_dict,
                                    project_name = project.value,
                                    n_frames = 10)

WidgetMaker(children=(IntText(value=0, description='Number of workflows:', style=DescriptionStyle(description_…

FloatSlider(value=0.8, continuous_update=False, description='Aggregation threshold:', max=1.0, readout_format=…

IntSlider(value=3, continuous_update=False, description='Min numbers of users:', max=15, min=1, style=SliderSt…

FileChooser(path='.', filename='', title='HTML(value='<b>Choose location to store frames</b>')', show_hidden='…

Output()

...

/cephyr/NOBACKUP/groups/snic2021-6-9/koster_movies/970708 TMBL-ROV 1997 Tape 2 S Kostergrund.mov


100%|##########| 160M/160M [00:40<00:00, 3.92Mb/s] 
  0%|          | 32.8k/155M [00:00<10:41, 242kb/s]

/cephyr/NOBACKUP/groups/snic2021-6-9/koster_movies/990810 TMBL-ROV 1999 Väderöarna Tape 46.mov


100%|##########| 155M/155M [00:36<00:00, 4.26Mb/s] 
/Users/jurie.germishuys/Workspace/odf/koster-uw/database/koster_lab_development/src/pims/pims/pyav_reader.py:232: MethodDeprecationWarning: VideoStream.seek is deprecated.
  self._stream.seek(timestamp + self._first_pts)


Frames extracted successfully


## List available frames to upload to Zooniverse

### Frame modifications

In [ ]:
# Work in progress
clip_modification = t4.select_modification()

In [ ]:
# Modify the clips
clips_ready_df = t4.modify_clips(clips_to_upload_df = clips_to_upload_df.reset_index(drop=True), 
                                 movie_i = movie_i.value,
                                 clip_modification = clip_modification.label, 
                                 modification_details = clip_modification.value)

In [ ]:
# Review the size of the modified clips
t4.check_clip_size(clip_paths = clips_ready_df["modif_clip_path"].unique())

In [ ]:
# Compare the original and modified clips
t4.compare_clips(df = clips_ready_df)

In [ ]:
### Frame modifications

# Work in progress
clip_modification = t4.select_modification()

# Modify the clips
clips_ready_df = t4.modify_clips(clips_to_upload_df = clips_to_upload_df.reset_index(drop=True), 
                                 movie_i = movie_i.value,
                                 clip_modification = clip_modification.label, 
                                 modification_details = clip_modification.value)

# Review the size of the modified clips
t4.check_clip_size(clip_paths = clips_ready_df["modif_clip_path"].unique())

# Compare the original and modified clips
t4.compare_clips(df = clips_ready_df)


### Preview the frames

In [108]:
# Compare the original and modified clips
t5.compare_frames(df = frames_to_upload_df)

Dropdown(description='Select original frame:', layout=Layout(width='50%'), options=('/Users/jurie.germishuys/W…

Output()

### Set Zooniverse metadata

In [116]:
upload_to_zoo, sitename, created_on = t5.set_zoo_metadata(df = frames_to_upload_df, species_list = species_i.value,
                                                          project_name = project.value, db_info_dict = db_info_dict)

#upload_to_zoo = upload_to_zoo.fillna("unknown")



### Upload frames to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [126]:
t5.upload_frames_to_zooniverse(upload_to_zoo = upload_to_zoo, 
                              sitename = sitename,
                              species_list = species_i.value,
                              created_on = created_on,
                              project = str(zoo_info_dict["subjects"]["project_id"].unique()[0]))

#END

  0%|          | 0/80 [00:00<?, ?it/s]

80_frames_Common sea pen_Kostergrund_8/07/1997 subject set created
uploading subjects to Zooniverse
/Users/jurie.germishuys/Workspace/odf/koster-uw/database/koster_lab_development/tutorials/temp_frames/970708 TMBL-ROV 1997 Tape 2 S Kostergrund_frame_27037_6.jpg
{'species_id': 6, 'movie_id': 40, 'filename': '970708 TMBL-ROV 1997 Tape 2 S Kostergrund.mov', 'siteName': 'Kostergrund', 'created_on': '8/07/1997', 'Author': 'Tomas Lundälv', 'fps': 25.0, 'duration': 4028.4, 'survey_start': 0, 'survey_end': 4028.4}


  1%|▏         | 1/80 [00:01<01:19,  1.01s/it]

subject saved
/Users/jurie.germishuys/Workspace/odf/koster-uw/database/koster_lab_development/tutorials/temp_frames/970708 TMBL-ROV 1997 Tape 2 S Kostergrund_frame_27062_6.jpg
{'species_id': 6, 'movie_id': 40, 'filename': '970708 TMBL-ROV 1997 Tape 2 S Kostergrund.mov', 'siteName': 'Kostergrund', 'created_on': '8/07/1997', 'Author': 'Tomas Lundälv', 'fps': 25.0, 'duration': 4028.4, 'survey_start': 0, 'survey_end': 4028.4}


  2%|▎         | 2/80 [00:01<01:09,  1.12it/s]

...

 99%|█████████▉| 79/80 [01:07<00:00,  1.20it/s]

subject saved
/Users/jurie.germishuys/Workspace/odf/koster-uw/database/koster_lab_development/tutorials/temp_frames/990810 TMBL-ROV 1999 Väderöarna Tape 46_frame_24450_6.jpg
{'species_id': 6, 'movie_id': 52, 'filename': '990810 TMBL-ROV 1999 Väderöarna Tape 46.mov', 'siteName': 'Vader islands', 'created_on': '10/08/1999', 'Author': 'Gothenburg University', 'fps': 50.0, 'duration': 301.44, 'survey_start': 0, 'survey_end': 301.44}


100%|██████████| 80/80 [01:08<00:00,  1.16it/s]

subject saved


Subjects uploaded to Zooniverse
